In [ ]:
# ===========================================
# GOOGLE COLAB SETUP
# ===========================================
# Run this cell FIRST if using Google Colab

import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    # Install dependencies
    !pip install -q aisuite openai duckdb pandas pydantic python-dotenv tavily-python wikipedia markdown

    # Clone the repo and navigate to workshop folder
    !git clone -b dev https://github.com/nestauk/discovery_residency_agentic_ai.git
    %cd discovery_residency_agentic_ai/discovery_residency_agentic_ai/evals_deep_dive

    # Set up API keys from Colab Secrets
    from google.colab import userdata
    import os
    os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
    os.environ['TAVILY_API_KEY'] = userdata.get('TAVILY_API_KEY')

    print("✅ Colab setup complete! API keys loaded from Secrets.")
else:
    print("Not running in Colab - skipping setup.")

# Evaluating AI Agents
## A Hands-On Workshop

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/nestauk/discovery_residency_agentic_ai/blob/dev/discovery_residency_agentic_ai/evals_deep_dive/workshop.ipynb)

Welcome! In this workshop, you'll learn how to design evaluations for AI agents.

![Agent Loop](https://github.com/nestauk/discovery_residency_agentic_ai/blob/dev/discovery_residency_agentic_ai/evals_deep_dive/assets/anthropic-agent-loop.webp?raw=1)

## Two "Axes" of Evaluation

When evaluating AI agents, we think along **two axes**:

*Inspired by [Andrew Ng's Agentic AI course](https://www.deeplearning.ai/short-courses/)*

### Axis 1: Per example ground truth?

- **Per example ground truth**: You know the correct answer for each test case
  - *Example: "Extract the date from this invoice" → expected: "2024-03-15"*
- **No per example ground truth**: There's no single "correct" answer
  - *Example: "Write marketing copy for this product"*

### Axis 2: Evaluate with code or LLM-as-judge?

- **Evaluate with code (objective)**: Code can verify correctness
  - *Example: `if extracted_date == actual_date: num_correct += 1`*
- **LLM-as-judge (subjective)**: Requires an LLM to evaluate quality
  - *Example: "Grade this chart according to whether it has clear axes labels..."*

### The Evaluation Quadrant

These two axes give us four quadrants — each with different evaluation strategies:

![Two Axes of Evaluation](https://github.com/nestauk/discovery_residency_agentic_ai/blob/dev/discovery_residency_agentic_ai/evals_deep_dive/assets/two-axes-of-evaluation.png?raw=1)

### Today's Focus

We'll explore **two quadrants** with hands-on examples:

**Top-left: SQL Agent**
- Has ground truth (we know the correct query results)
- Evaluate with code (compare actual vs expected)

**Bottom-right: Web Search Agent**
- No ground truth (no single "correct" research summary)
- LLM-as-judge (use a rubric to assess quality)

---
## Setup

Run this cell to import everything we need.

In [ ]:
# Environment
from dotenv import load_dotenv
load_dotenv()

from datetime import datetime
from aisuite import Client

# Utilities
from utils.helpers import make_verbose_tool
from utils.display import print_html, display_rubric_scores, display_test_results
from utils.evaluators import evaluate_sql_result, llm_judge

# Import the tool FUNCTION (aisuite calls this automatically)
from tools.research_tools import tavily_search_tool

# Initialize the LLM client
client = Client()

print("Setup complete!")

Setup complete!


### API Key Configuration

This workshop uses two external services:

| Service | Purpose | How to Get |
|---------|---------|------------|
| **Tavily** | Web search tool | Free at [tavily.com](https://tavily.com) |
| **OpenAI** | LLM for agents | [platform.openai.com](https://platform.openai.com/api-keys) |

**Local setup**: Create a `.env` file with your keys.

**Google Colab**: Use Colab Secrets (click the key icon in the left sidebar) to add:
- `OPENAI_API_KEY`
- `TAVILY_API_KEY`

Run the cell below to check your keys are configured:

In [ ]:
# Check API keys and detect which LLM provider to use
from utils.helpers import check_api_keys
MODEL = check_api_keys()

✅ TAVILY_API_KEY found
✅ OPENAI_API_KEY found → using openai:gpt-5-mini

✅ All API keys configured! You're ready to go.


---
## SQL Agent — Objective Evaluation

Let's start with the simplest type of evaluation: **objective with ground truth**.

We have a SQL agent that answers questions about a product nutrition database.

The evaluation is simple: **does the query return the expected result?**

### The Database

Our database contains nutrition information for food products:
- `product_name` - Product name
- `kcal_per_100g`, `protein_per_100g`, `sugar_per_100g`, etc.
- `npm_score` - Nutrient Profile Model score (UK HFSS metric)
- `converted_npm_score` - NPM score converted using Oxford method (npm_score*-2 + 70)

In [ ]:
# Set up the products database
from tools.sql_tools import db, setup_products_db

count = setup_products_db()
print(f"Database ready! {count} products loaded.")

# Quick peek at the data
db.execute("SELECT product_name, kcal_per_100g, protein_per_100g, sugar_per_100g FROM products LIMIT 5").fetchdf()

Database ready! 25 products loaded.


,product_name,kcal_per_100g,protein_per_100g,sugar_per_100g
0,Whole Milk,64.0,3.4,4.7
1,Greek Yogurt,97.0,9.0,3.6
2,Cheddar Cheese,416.0,25.4,0.1
3,White Bread,265.0,9.0,5.0
4,Wholemeal Bread,247.0,10.9,4.2


### The Tool

A **tool** is just a function that the LLM can call. Here's our SQL query tool:

In [ ]:
# ===========================================
# THE SQL QUERY TOOL
# ===========================================
# This is what the agent will use to execute SQL queries.
# A tool is just a function that takes arguments and returns results.

def sql_query_tool(query: str) -> list[dict]:
    """Execute a SQL query and return results as list of dicts."""
    try:
        result = db.execute(query).fetchdf()
        return result.to_dict(orient="records")
    except Exception as e:
        return [{"error": str(e)}]

print("Tool defined: sql_query_tool(query) -> list[dict]")

Tool defined: sql_query_tool(query) -> list[dict]


### Computing Ground Truth

Before we can test the agent, we need to know the **correct answers**.

Let's run SQL queries directly on the database to establish our ground truth values:

In [ ]:
# Q1: How many products are above the target converted NPM score threshold of 69?
sql_query_tool("""
    SELECT COUNT(*) as count
    FROM products
    WHERE converted_npm_score > 69
""")

[{'count': 9}]

In [ ]:
# Q2: What percentage of products are above the threshold?
sql_query_tool("""
    SELECT ROUND(100.0 * COUNT(*) FILTER (WHERE converted_npm_score > 69) / COUNT(*), 1) as pct
    FROM products
""")

[{'pct': 36.0}]

In [ ]:
# Q3: Which product has the highest absolute protein content?
sql_query_tool("""
    SELECT product_name, ROUND(protein_per_100g * grams / 100, 1) as total_protein_g
    FROM products
    ORDER BY total_protein_g DESC
    LIMIT 1
""")

[{'product_name': 'Porridge Oats', 'total_protein_g': 110.0}]

In [ ]:
# Q4: What is the average kcal_per_100g across all products?
sql_query_tool("""
    SELECT ROUND(AVG(kcal_per_100g)) as avg_kcal
    FROM products
""")

[{'avg_kcal': 281.0}]

In [ ]:
# Q5: HARDER - Which products are in the bottom 25% for healthiness but top 25% for protein?
sql_query_tool("""
    WITH ranked AS (
        SELECT product_name, converted_npm_score, protein_per_100g,
            NTILE(4) OVER (ORDER BY converted_npm_score ASC) as health_quartile,
            NTILE(4) OVER (ORDER BY protein_per_100g DESC) as protein_quartile
        FROM products
    )
    SELECT product_name, converted_npm_score, protein_per_100g
    FROM ranked
    WHERE health_quartile = 1 AND protein_quartile = 1
""")

[{'product_name': 'Cheddar Cheese',
  'converted_npm_score': 34,
  'protein_per_100g': 25.399999618530273}]

In [ ]:
# ===========================================
# THE SQL AGENT
# ===========================================

def sql_agent(question: str, model: str = MODEL) -> dict:
    """
    Answer questions about the products database using SQL.

    Returns:
        dict with 'answer' (extracted value) and 'raw_response' (full LLM output)
    """
    print(f"Question: {question}")

    prompt = f"""
    You have access to a SQL database with a 'products' table containing food nutrition data.

    Columns: product_name, grams, standardised_volume, kcal_per_100g, fat_per_100g,
    satfat_per_100g, protein_per_100g, carbs_per_100g, sugar_per_100g,
    sodium_per_100g, salt_per_100g, fibre_per_100g, npm_score, converted_npm_score

    Answer this question: {question}

    Instructions:
    1. Write a SQL query to get the answer
    2. Execute it using sql_query_tool
    3. Return ONLY the final answer (a number, name, or short phrase)
    """.strip()

    tools = [make_verbose_tool(sql_query_tool, "database")]

    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        tools=tools,
        tool_choice="auto",
        max_turns=3,
    )

    answer = response.choices[0].message.content
    print(f"Answer: {answer}")
    return {"answer": answer, "raw_response": response}

print("SQL agent defined!")

SQL agent defined!


### Try it yourself!

Test the agent with your own questions about the products database:

In [ ]:
NATURAL_LANGUAGE_QUERY = "Which product has the highest nutrient density across all measures?"

result = sql_agent(NATURAL_LANGUAGE_QUERY)

Question: Which product has the highest nutrient density across all measures?
  [database] Calling tool...
  [database] query: SELECT product_name, COALESCE(converted_npm_score, npm_score) AS score
FROM products
ORDER BY score DESC
LIMIT 1;
  [database] Returned 1 results
Answer: Chicken Breast


### Testing the Agent

Now that we have ground truth, let's see if the agent can figure out the correct SQL queries on its own.

We'll give it the same questions in natural language and check if its answers match:

In [ ]:
# ===========================================
# EVALUATION TEST CASES
# ===========================================
# Each test case has a question and the expected answer

TEST_CASES = [
    {
        "question": "What pct of products are above the target converted npm score threshold of 69?",
        "expected": "36",
        "comparison": "contains"
    },
    {
        "question": "Which product has the highest absolute protein content, and how much is it?",
        "expected": "Porridge Oats",
        "comparison": "contains"
    },
    {
        "question": "What is the average kcal_per_100g across all products? Round to the nearest whole number.",
        "expected": "281",
        "comparison": "contains"
    },
    {
        "question": "Which products are in the bottom 25% for healthiness but top 25% for protein content?",
        "expected": "Cheddar Cheese",
        "comparison": "contains"
    },
]

print(f"Defined {len(TEST_CASES)} test cases")

Defined 4 test cases


In [ ]:
# Run all test cases
print("Running all test cases...\n")

results = []
for test in TEST_CASES:
    result = sql_agent(test["question"])
    passed, _ = evaluate_sql_result(
        result["answer"],
        test["expected"],
        test["comparison"]
    )
    results.append({
        "question": test["question"],
        "expected": test["expected"],
        "actual": result["answer"],
        "passed": passed
    })
    print()

# Display results
display_test_results(results, title="SQL Agent Evaluation")

Running all test cases...

Question: What pct of products are above the target converted npm score threshold of 69?
  [database] Calling tool...
  [database] query: SELECT ROUND(100.0 * SUM(CASE WHEN converted_npm_score > 69 THEN 1 ELSE 0 END) / COUNT(*), 2) AS pct_above_69 FROM products;
  [database] Returned 1 results
Answer: 36.0%

Question: Which product has the highest absolute protein content, and how much is it?
  [database] Calling tool...
  [database] query: SELECT product_name, (protein_per_100g * grams / 100.0) AS protein_grams
FROM products
ORDER BY protein_grams DESC
LIMIT 1;
  [database] Returned 1 results
Answer: Porridge Oats — 110.0 g

Question: What is the average kcal_per_100g across all products? Round to the nearest whole number.
  [database] Calling tool...
  [database] query: SELECT ROUND(AVG(kcal_per_100g)) AS avg_kcal FROM products;
  [database] Returned 1 results
Answer: 281

Question: Which products are in the bottom 25% for healthiness but top 25% for protei

Question,Expected,Result
What pct of products are above the target converted npm score threshold of 69?,36,✓ 36.0%
"Which product has the highest absolute protein content, and how much is it?",Porridge Oats,✓ Porridge Oats — 110.0 g
What is the average kcal_per_100g across all products? Round to the nearest whole number.,281,✓ 281
Which products are in the bottom 25% for healthiness but top 25% for protein content?,Cheddar Cheese,"✗ Butter, Olive Oil"


---
### Key Takeaway: SQL Agent Evaluation

The SQL agent evaluation is **objective with ground truth**:
- We know the correct answer
- We can check it with code
- No ambiguity about pass/fail

**But what about subjective qualities?**

For these, we'd need **LLM-as-judge** evaluation.

Let's test that out by looking at a more complex agent where evaluation isn't so clear-cut...

---
## The Research Agent

We have a research agent that searches the web for information on any topic.

### How it works

The agent uses an LLM to call a tool:
- **tavily_tool** - general web search  

**You could design your own tools and plug them in!**

In [ ]:
# ===========================================
# THE RESEARCH AGENT
# ===========================================
# This function orchestrates an LLM with a web search tool to research any topic.
# You could modify this to use different tools or prompts.

def research_agent(topic: str, model: str = MODEL) -> str:
    """
    Research a topic using web search.

    Args:
        topic: The topic to research
        model: The LLM to use for orchestration

    Returns:
        Research results with sources and URLs
    """
    print(f"Researching: {topic}")

    prompt = f"""
    You are a research assistant. Use tavily_search_tool ONCE to search for information,
    then immediately provide a summary based on what you found.

    Topic to research: {topic}

    Instructions:
    1. Search once for the topic
    2. Summarise the key findings (2-3 paragraphs)
    3. List the source URLs at the end

    Today's date is {datetime.now().strftime('%Y-%m-%d')}.
    """.strip()

    tools = [make_verbose_tool(tavily_search_tool, "web")]

    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        tools=tools,
        tool_choice="auto",
        max_turns=2,  # 1 search + 1 response
    )

    print("Done!")
    return response.choices[0].message.content

print("Research agent defined!")

Research agent defined!


### Try it yourself

Let's test our research agent.

In [ ]:
# ===========================================
# CHANGE THIS to research your own topic
# ===========================================
TOPIC = "recent developments in heat pump policy in the UK"

print(f"Topic: {TOPIC}")

Topic: recent developments in heat pump policy in the UK


In [ ]:
# Run the research agent (this may take 10-30 seconds)
research_output = research_agent(TOPIC)

print_html(research_output, title=f"Research Results: {TOPIC}")

Researching: recent developments in heat pump policy in the UK
  [web] Calling tool...
  [web] query: recent developments in heat pump policy in the UK 2024 2025 government policy incentives heat pump grants boiler upgrade scheme heat pump rollout 2025 November
  [web] max_results: 5
  [web] include_images: False
  [web] Returned 5 results
Done!


---
## LLM-as-Judge Evaluation

Some qualities are **subjective** and hard to check with code:
- Is the writing clear and well-organised?
- Are the sources relevant to the query?
- Is the information comprehensive?

**Solution**: Use another LLM as a "judge" to score the output against a rubric.

### The Judge Prompt

This is the actual prompt sent to the judge LLM. You can modify it!

### How it works

1. Define a **rubric** with scoring dimensions
2. Build a **prompt** that asks the LLM to score each dimension
3. Parse the structured JSON response

In [ ]:
# ===========================================
# THE JUDGE PROMPT
# ===========================================
# This is the prompt sent to the judge LLM. Feel free to modify it.
# - Change the calibration line to see how scores change
# - Add more specific instructions
# - Change the scoring scale

JUDGE_PROMPT_TEMPLATE = """You are an evaluation judge. Score the following output on each dimension.

## Output to Evaluate:
{output}

## Scoring Rubric (score each 1-5):
{rubric_text}

## Instructions:
For each dimension, provide:
1. A score from 1 (poor) to 5 (excellent)
2. A brief explanation (1-2 sentences)

Be critical. A score of 5 should be rare.

Respond in JSON format:
{{
    "scores": {{{score_template}}},
    "explanations": {{{explanation_template}}}
}}
"""

print("Judge prompt template defined!")

Judge prompt template defined!


### Activity: Design a Rubric

A rubric defines what dimensions to evaluate. **This is a design choice** — there's no single "right" rubric!

We'll start with one criterion, then you'll add two more based on what *you* think matters.

In [ ]:
# ===========================================
# ACTIVITY: Design your rubric
# ===========================================
# We've started with ONE criterion.
# Ask yourself: "If I were grading this summary, what would I look for?"

RUBRIC = {
    "coherence": "Does the summary flow logically? Are ideas connected with smooth transitions?",
    # ADD 2 MORE CRITERIA BELOW
    # Examples to consider:
       "clarity": "Is the writing clear and easy to understand?",
    #   "completeness": "Does it cover the key aspects of the topic?",
       "relevance": "Does it stay focused on the research question?",
    #   "accuracy": "Are claims supported by the sources cited?",
       "conciseness": "Is it appropriately brief without unnecessary filler?",
    "persuasiveness":"Does the article convince the user of a point of view?",
}

print(f"Rubric has {len(RUBRIC)} dimension(s)")
for dim, desc in RUBRIC.items():
    print(f"  - {dim}: {desc}")

Rubric has 5 dimension(s)
  - coherence: Does the summary flow logically? Are ideas connected with smooth transitions?
  - clarity: Is the writing clear and easy to understand?
  - relevance: Does it stay focused on the research question?
  - conciseness: Is it appropriately brief without unnecessary filler?
  - persuasiveness: Does the article convince the user of a point of view?


In [ ]:
# Build the prompt from template + rubric + output
rubric_text = "\n".join(f"- **{dim}**: {desc}" for dim, desc in RUBRIC.items())
score_template = ", ".join(f'"{dim}": <score>' for dim in RUBRIC.keys())
explanation_template = ", ".join(f'"{dim}": "<explanation>"' for dim in RUBRIC.keys())

judge_prompt = JUDGE_PROMPT_TEMPLATE.format(
    output=research_output,
    rubric_text=rubric_text,
    score_template=score_template,
    explanation_template=explanation_template
)

# Run the LLM judge evaluation
judge_result = llm_judge(judge_prompt, RUBRIC)

# Display the results
display_rubric_scores(judge_result, title="LLM Judge Evaluation")

---
## Human Review Patterns

Sometimes you need **human judgment** that can't be automated:
- Is this factually accurate? (requires domain expertise)
- Is this appropriate for our use case?
- Would a user find this helpful?

### When to use each method?

| Method | Use When |
|--------|----------|
| **Automated** | Objective, code-checkable criteria |
| **LLM-as-judge** | Subjective but definable in a rubric |
| **Human review** | Domain expertise required, high stakes |

### Example: Human Review Checklist

```python
HUMAN_REVIEW_CHECKLIST = {
    "factual_accuracy": {
        "question": "Are all factual claims accurate?",
        "type": "yes_no_unsure",
    },
    "source_verification": {
        "question": "Did you spot-check at least one source URL?",
        "type": "yes_no",
    },
    "user_helpful": {
        "question": "Would this be helpful to someone researching this topic?",
        "type": "scale_1_5",
    },
}
```

### Key principles for human review

1. **Structure the review** - Don't ask "is this good?", ask specific questions
2. **Use checklists** - Binary yes/no questions are faster and more consistent
3. **Provide context** - Show reviewers what they need to evaluate
4. **Collect rationale** - Ask reviewers to explain their judgments

---
## Putting It Together

In practice, you'll often combine multiple evaluation types:

```
Agent Output
    │
    ▼
┌─────────────────┐
│ Objective Checks│ ──► Fast, cheap, catches obvious issues
└────────┬────────┘
         │
         ▼
┌─────────────────┐
│   LLM Judge     │ ──► Scalable quality assessment  
└────────┬────────┘
         │
         ▼
┌─────────────────┐
│  Human Review   │ ──► High-stakes verification (sample)
└─────────────────┘
```

### The Key Insight

**Layer your evaluations by cost and coverage:**

| Method | Cost | Coverage | Best For |
|--------|------|----------|----------|
| Objective checks | Low | 100% | Catching obvious failures |
| LLM-as-judge | Medium | 100% | Quality assessment at scale |
| Human review | High | Sample | Validation, edge cases |

---
## Key Takeaways

1. **Component-level evals** let you test individual pieces of an AI system

2. **Objective evaluations** (ground truth) are fast, cheap, reproducible

3. **LLM-as-judge** scales subjective evaluation but requires rubric design

4. **Human review** is essential for high-stakes and domain expertise

5. **Combine methods** for robust evaluation pipelines

---
## Activity: Design Evals for a Comms Agent

Let's apply what we've learned to a real-world Nesta project.

### The Problem

We're building an agentic AI solution to [assist heat pump installers with their admin workload](https://www.nesta.org.uk/project/showing-the-practical-value-of-agentic-ai-for-driving-impact/).

The **Communications Agent** helps interpret and reply to incoming emails and documents about grid connection applications.

### Agent Capabilities

1. **Triage emails** — Categorise incoming messages by action required
2. **Extract key information from PDFs** — Pull out relevant data from attached documents
3. **Summarise and present context** — Give human reviewers the information they need

**Your task**: In small groups, brainstorm evaluation strategies for each capability.

### Think About: The Evaluation Quadrant

For each capability, consider:
- **Do we have ground truth?** Can we know the "correct" answer in advance?
- **Can code check it?** Or do we need LLM-as-judge / human review?

| Capability | Ground Truth? | Eval Method? |
|------------|---------------|--------------|
| Email triage | ? | ? |
| PDF extraction | ? | ? |
| Summarise context | ? | ? |

### Capability 1: Email Triage

The agent categorises emails into one of three actions:
- **Ignore** — No action needed (spam, irrelevant, already handled)
- **Notify** — Flag for human attention but no response needed
- **Respond** — Needs a reply

**Context**: These are emails from a Distribution Network Operator (DNO) to heat pump installers regarding grid connection applications.

**Email A:**
> From: connections@ukpowernetworks.co.uk  
> Subject: Application HP-2024-78432 - Additional information required
>
> Dear Installer,
>
> We are processing your G99 application for the property at 14 Willow Lane, Cambridge, CB4 1AB.
>
> Before we can proceed, please provide:
> - Single line diagram showing isolation points
> - Confirmation of maximum export capacity (kW)
>
> Please respond within 10 working days or your application may be closed.
>
> Regards,  
> UK Power Networks Connections Team

---

**Email B:**
> From: no-reply@ukpowernetworks.co.uk  
> Subject: Planned maintenance notification - Eastern region
>
> This is an automated message.
>
> Planned maintenance will take place in the Eastern region on 15th December 2024 between 02:00-05:00. Some online services may be unavailable during this period.
>
> No action is required.
>
> UK Power Networks

---

**Email C:**
> From: connections@ukpowernetworks.co.uk  
> Subject: Re: HP-2024-81205 - Application approved
>
> Dear Installer,
>
> Your G99 application for 27 Oak Street, Norwich, NR1 3PQ has been approved.
>
> Please find attached your connection offer and acceptance form. The offer is valid for 90 days.
>
> If you have any questions, please contact us quoting your reference number.
>
> Best regards,  
> Connections Team

---

**Write your labels:**
```
Email A:  [ ] Ignore   [ ] Notify   [ ] Respond

Email B:  [ ] Ignore   [ ] Notify   [ ] Respond

Email C:  [ ] Ignore   [ ] Notify   [ ] Respond
```

**Discussion**: Did everyone agree? What context would change your answer?

---

**Design the eval:**
- Does this have ground truth?
- Can code check it, or do you need LLM-as-judge?
- What test cases would you create?

### Capability 2: PDF Information Extraction

The agent reads attached documents and extracts key information: addresses, quote amounts, and flags any discrepancies.

**Context**: Heat pump installers receive quotes from suppliers. The agent should extract structured data and catch errors.

**Your task**: Review this quote and extract the required fields. Can you spot the discrepancy?

<div style="font-family: Arial, sans-serif; max-width: 600px; border: 1px solid #ccc; padding: 24px; background: #fff; margin: 16px 0;">

<div style="display: flex; justify-content: space-between; align-items: start; margin-bottom: 20px;">
<div>
<div style="font-size: 20px; font-weight: bold; color: #1e40af;">HeatFlow Supplies Ltd</div>
<div style="font-size: 12px; color: #666;">Unit 7, Riverside Industrial Estate<br>Norwich, NR3 1QS<br>VAT: GB 123 4567 89</div>
</div>
<div style="text-align: right;">
<div style="font-size: 24px; font-weight: bold; color: #333;">QUOTATION</div>
<div style="font-size: 12px; color: #666;">Quote #: HF-Q-2024-0892<br>Date: 22 Nov 2024<br>Valid until: 22 Dec 2024</div>
</div>
</div>

<div style="background: #f8fafc; padding: 12px; margin-bottom: 20px; border-radius: 4px;">
<div style="font-size: 11px; color: #666; text-transform: uppercase;">Quote For:</div>
<div style="font-weight: 600;">GreenHeat Installations Ltd</div>
<div style="font-size: 13px;">47 Churchill Road<br>Ipswich, Suffolk<br>IP4 2QT</div>
</div>

<div style="background: #f8fafc; padding: 12px; margin-bottom: 20px; border-radius: 4px;">
<div style="font-size: 11px; color: #666; text-transform: uppercase;">Installation Address:</div>
<div style="font-size: 13px;">14 Willow Lane<br>Cambridge<br>CB4 1AB</div>
</div>

<table style="width: 100%; border-collapse: collapse; font-size: 13px; margin-bottom: 20px;">
<thead>
<tr style="background: #1e40af; color: white;">
<th style="padding: 10px; text-align: left;">Description</th>
<th style="padding: 10px; text-align: center;">Qty</th>
<th style="padding: 10px; text-align: right;">Unit Price</th>
<th style="padding: 10px; text-align: right;">Amount</th>
</tr>
</thead>
<tbody>
<tr style="border-bottom: 1px solid #e5e7eb;">
<td style="padding: 10px;">Vaillant Arotherm Plus 7kW Heat Pump</td>
<td style="padding: 10px; text-align: center;">1</td>
<td style="padding: 10px; text-align: right;">£4,250.00</td>
<td style="padding: 10px; text-align: right;">£4,250.00</td>
</tr>
<tr style="border-bottom: 1px solid #e5e7eb;">
<td style="padding: 10px;">Vaillant uniTOWER 190L Cylinder</td>
<td style="padding: 10px; text-align: center;">1</td>
<td style="padding: 10px; text-align: right;">£1,850.00</td>
<td style="padding: 10px; text-align: right;">£1,850.00</td>
</tr>
<tr style="border-bottom: 1px solid #e5e7eb;">
<td style="padding: 10px;">Installation Kit & Refrigerant Lines (15m)</td>
<td style="padding: 10px; text-align: center;">1</td>
<td style="padding: 10px; text-align: right;">£420.00</td>
<td style="padding: 10px; text-align: right;">£420.00</td>
</tr>
<tr style="border-bottom: 1px solid #e5e7eb;">
<td style="padding: 10px;">Delivery & Handling</td>
<td style="padding: 10px; text-align: center;">1</td>
<td style="padding: 10px; text-align: right;">£85.00</td>
<td style="padding: 10px; text-align: right;">£85.00</td>
</tr>
<tr style="border-bottom: 1px solid #e5e7eb;">
<td style="padding: 10px;">Delivery & Handling</td>
<td style="padding: 10px; text-align: center;">1</td>
<td style="padding: 10px; text-align: right;">£85.00</td>
<td style="padding: 10px; text-align: right;">£85.00</td>
</tr>
</tbody>
</table>

<div style="display: flex; justify-content: flex-end;">
<table style="font-size: 13px; min-width: 200px;">
<tr>
<td style="padding: 6px 12px;">Subtotal:</td>
<td style="padding: 6px 12px; text-align: right;">£6,690.00</td>
</tr>
<tr>
<td style="padding: 6px 12px;">VAT (20%):</td>
<td style="padding: 6px 12px; text-align: right;">£1,338.00</td>
</tr>
<tr style="font-weight: bold; font-size: 15px; border-top: 2px solid #1e40af;">
<td style="padding: 10px 12px;">Total:</td>
<td style="padding: 10px 12px; text-align: right;">£8,028.00</td>
</tr>
</table>
</div>

<div style="margin-top: 24px; padding-top: 16px; border-top: 1px solid #e5e7eb; font-size: 11px; color: #666;">
<strong>Terms:</strong> Payment due within 30 days of invoice. Prices valid for 30 days from quote date.<br>
<strong>Lead time:</strong> 5-7 working days from order confirmation.
</div>

</div>

**Extract the following:**
```
Installation address:  _______________________________________________

Quote total (inc VAT): _______________________________________________

Discrepancy found:     _______________________________________________
```

**Design the eval:**
- Does this have ground truth?
- Can code check it, or do you need LLM-as-judge?
- How would you test discrepancy detection?

### Capability 3: Summarise and Present Context

The agent creates summaries that give human reviewers the context they need to make decisions.

**Context**: The installer received the DNO email (Email A) requesting additional information. The agent should summarise this for the installer to action.

**Your task**: Review these 3 agent-generated summaries and rank them.

**Summary A:**
> UK Power Networks sent an email about application HP-2024-78432.

---

**Summary B:**
> **Action required**: DNO requesting additional documents for 14 Willow Lane, Cambridge (HP-2024-78432).
>
> They need:
> - Single line diagram with isolation points
> - Max export capacity confirmation
>
> **Deadline**: 10 working days or application may be closed.

---

**Summary C:**
> You have received correspondence from UK Power Networks regarding your G99 grid connection application. The application reference number is HP-2024-78432 and it pertains to the property located at 14 Willow Lane, Cambridge, CB4 1AB. The Distribution Network Operator has indicated that they require additional documentation before they can continue processing your application. Specifically, they have requested that you provide a single line diagram that clearly shows the isolation points, as well as confirmation of the maximum export capacity measured in kilowatts (kW). It is important to note that you should respond to this request within a timeframe of 10 working days. If you do not respond within this period, there is a possibility that your application may be closed.

---

**Rank them (1 = best, 3 = worst):**
```
Summary A:  Rank ___   Why? ________________________________________

Summary B:  Rank ___   Why? ________________________________________

Summary C:  Rank ___   Why? ________________________________________
```

**Design the eval:**
- Does this have ground truth?
- Can code check it, or do you need LLM-as-judge?
- What rubric criteria would you use?

### Group Discussion: Share Your Ideas

After brainstorming, let's share:

1. **Which capabilities are easiest to evaluate? Why?**

2. **Which need human review? What expertise is required?**

3. **What surprised you about designing these evals?**

4. **What would you prioritise building first?**

---
## Next Steps

To build a proper evaluation suite:

1. **Create an eval set**: 10-20 diverse test cases
2. **Define ground truth**: What should pass/fail?
3. **Run evals regularly**: Track metrics over time
4. **Iterate**: Improve your agent based on results

---
## Thank You!

Learn more about the agentic AI residency:

- **[Nesta welcomes AI agents resident](https://www.nesta.org.uk/project-updates/nesta-welcomes-ai-agents-resident/)**

- **[Showing the practical value of agentic AI for driving impact](https://www.nesta.org.uk/project/showing-the-practical-value-of-agentic-ai-for-driving-impact/)**

- **[The challenges of DNO applications for heat pump installers](https://www.nesta.org.uk/project-updates/the-challenges-of-distribution-network-operator-applications-for-heat-pump-installers/)**

Questions? Let's discuss!